# Functions

In [ ]:
pip install python-louvain

### Opinion Dynamics algorithm
Functions that define the algorithm and the opinion dynamics.

In [ ]:
import math
import numpy as np

# This function adds one new node in the network provided.
# The new node attaches to m nodes, and the probability to attach to each node is defined by the morphogenesis function (takes into account homophily / beta and degree of node)
# graph: The existing homophilial-BA graph. The nodes must have an "opinion" attribute.
# beta: Coefficient tuning the strength of the homophily effect in the network
# m: Number of edges to attach from a new node to existing nodes
def grow_homophilial_barabasi_albert_graph(graph, beta, m, current_iter):

    # Get existing opinions
    opinions = ([attributes['opinion'] for node, attributes in graph.nodes(data=True) if 'opinion' in attributes])

    # Generate a random opinion for the new node
    new_opinion = np.random.rand()

    #Creation of a list with the connection probabilities for the new node
    pp=[graph.degree(i) * math.exp(-beta*math.fabs(new_opinion - opinions[i])) for i in range(len(opinions))]

    #List of the nodes
    listNodes = [i for i in range(len(opinions))]

    newNodeIndex = len(listNodes)
    #Loop on the m new links
    for it in range(0, m):
        #Choice of the neighbour
        toLink=roulette_wheel_selection (pp)
        #Add the new link
        graph.add_edge(newNodeIndex, listNodes[toLink])  #roulette_wheel_selection returns the index
        #Cancel the selected node from the available pool for new links creation
        del pp[toLink]
        del listNodes [toLink]

    # Add random opinion (between 0 and 1) to new node
    graph.nodes[newNodeIndex]['opinion'] = new_opinion
    graph.nodes[newNodeIndex]['spawn_time'] = current_iter

    #print(f"The new node has edges to {graph.edges(newNodeIndex)} and an opinion {new_opinion}")



#################################################################
# Definition of the function for the model update. #
# Input: number of agents (nNodes), tolerance parameter
# (epsilon), number of iterations(NITER) #
#################################################################
import random
import pandas as pd
#import BoundedConfidence as BC
#import networkGenerationModel as NGM

# Opinion Dynamics algorithm
# graph: Scale-Free Network with nodes. Nodes must contain the atrribute 'opinion' with a numerical value.
# epsilon: Tolerance threshold
# T: Number of maximum iterations of the algorithm.
# beta: Coefficient tuning the strength of the homophily effect in the network
# m: Number of edges to attach from a new node to existing nodes
# delta: Coefficient tuning the probability of adding a new node to the network at each step of the algorithm
# num_initial_nodes: Number of nodes that existed in the network when it was created (at time = 0)
# mu: Regulates how closer two opinions get when they interact.
def growing_opinion_dynamics(graph, epsilon, T=1000, beta=0, m=3, delta=1, num_initial_nodes=5, mu = 0.5):

  deffuant_opinions = []

  iter = 0
  iter_integer = 0

  while iter < T:

    node1 = random.randint(0, graph.number_of_nodes() - 1)
    node1_neighbors = list(graph.neighbors(node1))
    node2 = random.choice(node1_neighbors)

    if np.fabs(graph.nodes[node1]['opinion'] - graph.nodes[node2]['opinion']) < epsilon:
        #Update opinions
        new_opinion1 = graph.nodes[node1]['opinion'] + mu*(graph.nodes[node2]['opinion'] - graph.nodes[node1]['opinion'])
        new_opinion2 = graph.nodes[node2]['opinion'] + mu*(graph.nodes[node1]['opinion'] - graph.nodes[node2]['opinion'])
        graph.nodes[node1]['opinion'] = new_opinion1
        graph.nodes[node2]['opinion'] = new_opinion2

    # Grow the BA network
    new_node_probability = delta / graph.number_of_nodes()
    random_number = random.random()
    if (random_number <= new_node_probability):
        grow_homophilial_barabasi_albert_graph(graph, beta, m, math.floor(iter))

    # We increase "iter" by a small decimal value.
    iter += 1 / graph.number_of_nodes()
    # math.floor(iter) returns the integer part of "iter". If it is higher than "iter_integer", it means we've
    # advanced enough to capture results (we have advanced one full unit of time)
    if math.floor(iter) > iter_integer:
        deffuant_opinions.append([attributes['opinion']
                                  for node, attributes in graph.nodes(data=True)
                                  if 'opinion' in attributes])
        iter_integer = math.floor(iter)   # Update "iter_integer" to the new integer iteration value.

  return deffuant_opinions

### Utilities
Functions that complement the algorithm and simulations.

NOTE: The two following functions are taken from: Gargiulo, F. & Gandica, Y. (2017). The Role of Homophily in the Emergence of Opinion Controversies. Journal of Artificial Societies and Social Simulation 20(3) 8, 2017 Doi: 10.18564/jasss.3448

In [ ]:
import networkx as nx
import numpy as np

# Function calculating the cumulative sum of a list
def accumu(lis):
    total = 0
    for x in lis:
        total += x
        yield total

# Function implementing the roulette-wheel-selection
def roulette_wheel_selection(ll):
    norm = sum(ll)
    normed_ll= [i / norm for i in ll]
    #Cumulative sum of the normalized probability vector: a sequence of newNode-1 numbers
    #from 0 to 1, separated by a distance proportional to the pp values
    accList=list(accumu(normed_ll))
    #Pick a random number
    nn=random.uniform(0,1)
    #Find where this number is located in the accumulation list: the position of nn in accList is the
    #node that will be chosen for connection
    position = 0
    while accList[position] < nn:
        position = position +1

    return position

In [ ]:
import networkx as nx
import numpy as np

# Identify hub nodes in a network
# Returns a dict item containing entries with only hub nodes.
def get_hub_nodes(graph):
    # Calcular el grado de cada nodo
    degrees = dict(graph.degree())

    # Opción 1: Considerar como hubs a los nodos con un grado mayor que un umbral específico, p.ej., 4 desviaciones estándar por encima de la media
    mean_degree = np.mean(list(degrees.values()))
    std_degree = np.std(list(degrees.values()))
    threshold = mean_degree + 4 * std_degree

    hubs = {node: deg for node, deg in degrees.items() if deg > threshold}
    return hubs

# These functions initialize different types of graphs.
def random_scale_free_graph(n):
  graph = nx.scale_free_graph(n)
  random_values = np.random.rand(n)
  for node in graph.nodes:
    graph.nodes[node]['opinion'] = random_values[node]
    graph.nodes[node]['spawn_time'] = 0
  return graph

def random_barabasi_albert_graph(n, m):
  ba_graph = nx.barabasi_albert_graph(n, m)
  random_values = np.random.rand(n)
  for node in ba_graph.nodes:
    ba_graph.nodes[node]['opinion'] = random_values[node]
    ba_graph.nodes[node1]['spawn_time'] = 0
  return ba_graph

def random_fully_connected_graph(n):
    random_values = np.random.rand(n)
    graph = nx.Graph()
    for node1 in range(0,n):
        for node2 in range(node1+1, n):
            graph.add_edge(node1,node2)
        graph.nodes[node1]['opinion'] = random_values[node1]
        graph.nodes[node1]['spawn_time'] = 0
    return graph

### Result visualization
Functions to show the simulations in different types of charts.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D

#-- En results_df, cada nodo es una columna, y las filas son los instantes de tiempo.
#-- Entonces, si sabemos que el nodo X es hub (porque lo vemos en el grafo), la columna X de results_df es de un nodo hub.

def deffuant_model_plot(graph, results_df, epsilon, m, beta, delta, num_initial_nodes):
    # Número de filas en el DataFrame
    x = range(results_df.shape[0])  # Usar range con el número de filas

    # Hub nodes
    hubs = get_hub_nodes(graph)

    for node, degree in hubs.items():
        neighbors = list(graph.neighbors(node))
        # Calcular la diferencia absoluta entre la opinión del nodo y la de sus vecinos
        filtered_differences = [
            abs(graph.nodes[node]["opinion"] - graph.nodes[neighbor]["opinion"])
            for neighbor in neighbors
            if abs(graph.nodes[node]["opinion"] - graph.nodes[neighbor]["opinion"]) < epsilon
        ]
        # Calcular el promedio de las diferencias
        average_difference = sum(filtered_differences) / len(filtered_differences) if filtered_differences else 0

        #print(f"Hub node {node} with degree {degree}, opinion {results_df[node].iloc[-1]} and avg. distance to neighbors (that can join opinions): {average_difference}")


    fig, ax = plt.subplots(figsize=(16, 9))

    # Obtener el máximo y mínimo valor de todo el DataFrame para normalizar los colores
    vmin = results_df.min().min()
    vmax = results_df.max().max()

    # Graficar cada columna del DataFrame como un gráfico de puntos (no hubs)
    for column in results_df.columns:
        if column not in hubs:  # Si el nodo no es un hub
            colors = results_df[column]  # Utilizar los valores de la columna actual para el color
            ax.scatter(x, results_df[column], label=column, c=colors, cmap='viridis', s=0.25, vmin=vmin, vmax=vmax, alpha=0.2)

    # Graficar los nodos hub para que aparezcan al frente
    for column in results_df.columns:
        if column in hubs:  # Si el nodo es un hub
            ax.scatter(x, results_df[column], label=column, color="#FF6F61", s=0.5, zorder=3, alpha = 0.4)

    # Añadir una barra de color para indicar la correspondencia de los colores con los valores
    scatter = ax.scatter([], [], c=[], cmap='viridis', s=5, vmin=vmin, vmax=vmax)  # Dummy scatter for colorbar
    cbar = plt.colorbar(scatter, ax=ax, orientation='vertical')
    cbar.set_label('Value')

    # Añadir etiquetas y título
    ax.set_xlabel('Time')
    ax.set_ylabel('Values')
    ax.set_title(f'Deffuant Model of a growing Homophilial Bar. Alb. Network')
    ax.set_xlim(0, len(results_df) - 1)

    # Crear un elemento de leyenda ficticio para "Hub node"
    hub_node_legend = Line2D([0], [0], marker='o', color='w', label='Hub node',
                             markerfacecolor='#FF6F61', markersize=10, linestyle='None')
    # Añadir la leyenda al gráfico
    ax.legend(handles=[hub_node_legend], loc='upper center', bbox_to_anchor=(0.5, -0.075))
    fig.tight_layout()

    # Añadir texto en la parte inferior del gráfico
    textstr = f'ε: {epsilon}  m: {m}  β: {beta}  δ: {delta}  N₀: {num_initial_nodes}'
    plt.gcf().text(0.5, -0.15, textstr, fontsize=12, ha='center', va='top', transform=ax.transAxes)

    return plt



# Visualización por puntos con gradiente de color
def deffuant_model_plot_highlight_hubs(graph, results_df, epsilon, m, beta, delta, num_initial_nodes):
    # Número de filas en el DataFrame
    x = range(results_df.shape[0])  # Usar range con el número de filas

    # Hub nodes
    hubs = get_hub_nodes(graph)

    #for node, degree in hubs.items():
    #    neighbors = list(graph.neighbors(node))
        # Calcular la diferencia absoluta entre la opinión del nodo y la de sus vecinos
    #    filtered_differences = [
    #        abs(graph.nodes[node]["opinion"] - graph.nodes[neighbor]["opinion"])
    #        for neighbor in neighbors
    #        if abs(graph.nodes[node]["opinion"] - graph.nodes[neighbor]["opinion"]) < epsilon
    #    ]
        # Calcular el promedio de las diferencias
    #    average_difference = sum(filtered_differences) / len(filtered_differences) if filtered_differences else 0

        #print(f"Hub node {node} with degree {degree}, opinion {results_df[node].iloc[-1]} and avg. distance to neighbors (that can join opinions): {average_difference}")


    fig, ax = plt.subplots(figsize=(16, 9))

    # Obtener el máximo y mínimo valor de todo el DataFrame para normalizar los colores
    vmin = results_df.min().min()
    vmax = results_df.max().max()

    # Obtener todos los vecinos de los hubs
    neighbors_of_hubs = set()
    for hub in hubs:
        neighbors_of_hubs.update(graph.neighbors(hub))

    # Graficar cada columna del DataFrame según las condiciones especificadas
    for column in results_df.columns:
        if column in hubs:  # Si el nodo es un hub
            ax.scatter(x, results_df[column], label=column, color="#FF6F61", s=0.1, zorder=3)
        elif column in neighbors_of_hubs:  # Si el nodo es vecino de un hub pero no es hub
            ax.scatter(x, results_df[column], label=column, color="#0BE59C", s=0.15, zorder=2)
        else:  # Si el nodo no es vecino de hub ni es hub
            ax.scatter(x, results_df[column], label=column, color="black", s=0.3, zorder=1)

    # Añadir etiquetas y título
    ax.set_xlabel('Time')
    ax.set_ylabel('Values')
    ax.set_title(f'Deffuant Model of a growing Homophilial Bar. Alb. Network')
    ax.set_xlim(0, len(results_df) - 1)

    # Crear un elemento de leyenda ficticio para "Hub node"
    hub_node_legend = Line2D([0], [0], marker='o', color='w', label='Hub node',
                             markerfacecolor='#FF6F61', markersize=10, linestyle='None')
    neighbor_node_legend = Line2D([0], [0], marker='o', color='w', label='Neighbor of Hub',
                                  markerfacecolor='#0BE59C', markersize=10, linestyle='None')
    non_hub_node_legend = Line2D([0], [0], marker='o', color='w', label='Non-Hub Node',
                                 markerfacecolor='black', markersize=10, linestyle='None')
    # Añadir la leyenda al gráfico
    ax.legend(handles=[hub_node_legend, neighbor_node_legend, non_hub_node_legend], loc='upper center', bbox_to_anchor=(0.5, -0.075))
    fig.tight_layout()

    # Añadir texto en la parte inferior del gráfico
    textstr = f'ε: {epsilon}  m: {m}  β: {beta}  δ: {delta}  N₀: {num_initial_nodes}'
    plt.gcf().text(0.5, -0.3, textstr, fontsize=12, ha='center', va='top', transform=ax.transAxes)

    return plt

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D

def deffuant_model_plot_highlight_communities(graph, results_df, epsilon, m, beta, delta, partition, num_initial_nodes):
    # Número de filas en el DataFrame
    x = range(results_df.shape[0])  # Usar range con el número de filas

    # Hub nodes
    hubs = get_hub_nodes(graph)

    for node, degree in hubs.items():
        neighbors = list(graph.neighbors(node))
        # Calcular la diferencia absoluta entre la opinión del nodo y la de sus vecinos
        filtered_differences = [
            abs(graph.nodes[node]["opinion"] - graph.nodes[neighbor]["opinion"])
            for neighbor in neighbors
            if abs(graph.nodes[node]["opinion"] - graph.nodes[neighbor]["opinion"]) < epsilon
        ]
        # Calcular el promedio de las diferencias
        average_difference = sum(filtered_differences) / len(filtered_differences) if filtered_differences else 0

        # print(f"Hub node {node} with degree {degree}, opinion {results_df[node].iloc[-1]} and avg. distance to neighbors (that can join opinions): {average_difference}")


    fig, ax = plt.subplots(figsize=(16, 9))

    # Obtener el máximo y mínimo valor de todo el DataFrame para normalizar los colores
    vmin = results_df.min().min()
    vmax = results_df.max().max()

    # Obtener todos los vecinos de los hubs
    neighbors_of_hubs = set()
    for hub in hubs:
        neighbors_of_hubs.update(graph.neighbors(hub))

    # Crear un mapa de colores para las comunidades
    num_comunidades = len(set(partition.values()))
    colormap = cm.get_cmap('tab10', num_comunidades)
    comunidad_colors = {comunidad: colormap(i) for i, comunidad in enumerate(set(partition.values()))}

    # Graficar cada columna del DataFrame según las condiciones especificadas
    for column in results_df.columns:
        comunidad = partition[column]
        color = comunidad_colors[comunidad]
        if column in hubs:  # Si el nodo es un hub
            ax.scatter(x, results_df[column], label=column, color=color, s=0.1, zorder=3)
        elif column in neighbors_of_hubs:  # Si el nodo es vecino de un hub pero no es hub
            ax.scatter(x, results_df[column], label=column, color=color, s=0.15, zorder=2)
        else:  # Si el nodo no es vecino de hub ni es hub
            ax.scatter(x, results_df[column], label=column, color=color, s=0.3, zorder=1)

    # Añadir etiquetas y título
    ax.set_xlabel('Time')
    ax.set_ylabel('Values')
    ax.set_title(f'Deffuant Model of a growing Homophilial Bar. Alb. Network')
    ax.set_xlim(0, len(results_df) - 1)

    # Crear leyenda personalizada
    legend_elements = []
    for comunidad, color in comunidad_colors.items():
        legend_elements.append(Line2D([0], [0], marker='o', color='w', label=f'Community {comunidad}',
                                      markerfacecolor=color, markersize=10, linestyle='None'))

    # Añadir la leyenda al gráfico
    ax.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.075))
    fig.tight_layout()

    # Añadir texto en la parte inferior del gráfico
    textstr = f'ε: {epsilon}  m: {m}  β: {beta}  δ: {delta}  N₀: {num_initial_nodes}'
    plt.gcf().text(0.5, -0.3, textstr, fontsize=12, ha='center', va='top', transform=ax.transAxes)

    return plt

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.lines import Line2D

def deffuant_model_plot_highlight_single_community(graph, results_df, epsilon, m, beta, delta, partition, community_number, num_initial_nodes):
    # Número de filas en el DataFrame
    x = range(results_df.shape[0])  # Usar range con el número de filas

    # Hub nodes
    hubs = get_hub_nodes(graph)

    for node, degree in hubs.items():
        neighbors = list(graph.neighbors(node))
        # Calcular la diferencia absoluta entre la opinión del nodo y la de sus vecinos
        filtered_differences = [
            abs(graph.nodes[node]["opinion"] - graph.nodes[neighbor]["opinion"])
            for neighbor in neighbors
            if abs(graph.nodes[node]["opinion"] - graph.nodes[neighbor]["opinion"]) < epsilon
        ]
        # Calcular el promedio de las diferencias
        average_difference = sum(filtered_differences) / len(filtered_differences) if filtered_differences else 0

    fig, ax = plt.subplots(figsize=(16, 9))

    # Obtener el máximo y mínimo valor de todo el DataFrame para normalizar los colores
    vmin = results_df.min().min()
    vmax = results_df.max().max()

    # Obtener todos los vecinos de los hubs
    neighbors_of_hubs = set()
    for hub in hubs:
        neighbors_of_hubs.update(graph.neighbors(hub))

    # Crear un mapa de colores para las comunidades
    comunidad_colors = {comunidad: 'gray' for comunidad in set(partition.values())}
    comunidad_colors[community_number] = 'yellow'  # Pintar la comunidad 11 en amarillo

    # Colores específicos para hubs y vecinos de hubs en la comunidad 11
    hub_color = '#FF6F61'
    neighbor_color = '#0BE59C'

    # Graficar cada columna del DataFrame según las condiciones especificadas
    for column in results_df.columns:
        comunidad = partition[column]
        if column in hubs and comunidad == community_number:  # Si el nodo es un hub en la comunidad 11
            color = hub_color
            ax.scatter(x, results_df[column], label=column, color=color, s=0.1, zorder=3)
        elif column in neighbors_of_hubs and comunidad == community_number:  # Si el nodo es vecino de un hub en la comunidad 11
            color = neighbor_color
            ax.scatter(x, results_df[column], label=column, color=color, s=0.15, zorder=2)
        else:  # Resto de nodos
            color = comunidad_colors[comunidad]
            ax.scatter(x, results_df[column], label=column, color=color, s=0.3, zorder=1)

    # Añadir etiquetas y título
    ax.set_xlabel('Time')
    ax.set_ylabel('Values')
    ax.set_title(f'Deffuant Model of a growing Homophilial Bar. Alb. Network')
    ax.set_xlim(0, len(results_df) - 1)

    # Crear leyenda personalizada
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label=f'Community {community_number} (Hubs)', markerfacecolor=hub_color, markersize=10, linestyle='None'),
        Line2D([0], [0], marker='o', color='w', label=f'Community {community_number} (Neighbors)', markerfacecolor=neighbor_color, markersize=10, linestyle='None'),
        Line2D([0], [0], marker='o', color='w', label=f'Community {community_number} (Others)', markerfacecolor='yellow', markersize=10, linestyle='None'),
        Line2D([0], [0], marker='o', color='w', label='Other Communities', markerfacecolor='gray', markersize=10, linestyle='None')
    ]

    # Añadir la leyenda al gráfico
    ax.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.075))
    fig.tight_layout()

    # Añadir texto en la parte inferior del gráfico
    textstr = f'ε: {epsilon}  m: {m}  β: {beta}  δ: {delta}  N₀: {num_initial_nodes}'
    plt.gcf().text(0.5, -0.3, textstr, fontsize=12, ha='center', va='top', transform=ax.transAxes)

    return plt


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx

def plot_degree_distribution_loglog(graphs, beta):
    # Obtener los grados de todos los nodos
    degrees = [degree for node, degree in graph.degree() for graph in graphs]

    # Normalizar los grados dividiéndolos por el máximo grado
    max_degree = max(degrees)
    normalized_degrees = [degree / max_degree for degree in degrees]

    # Calcular el histograma de los grados normalizados
    counts, bin_edges = np.histogram(normalized_degrees, bins=20, density=True)

    # Calcular el punto medio de cada bin para graficar
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

    # Filtrar para eliminar bines con frecuencia cero
    non_zero_mask = counts > 0
    filtered_bin_centers = bin_centers[non_zero_mask]
    filtered_counts = counts[non_zero_mask]

    # Aplicar logaritmo a los bin_centers y counts filtrados
    log_bin_centers = np.log(filtered_bin_centers)
    log_counts = np.log(filtered_counts)

    # Filtrar los datos para excluir los valores de X >= -0.5
    final_mask = log_bin_centers < -0.5
    final_log_bin_centers = log_bin_centers[final_mask]
    final_log_counts = log_counts[final_mask]

    # Ajustar una línea a los puntos filtrados
    slope, intercept = np.polyfit(final_log_bin_centers, final_log_counts, 1)

    # Crear un gráfico con escalas log-log usando plt.plot
    plt.figure(figsize=(10, 6))
    plt.plot(log_bin_centers, log_counts, marker='o', linestyle='-', color='blue', label='Histogram bins')
    plt.plot(final_log_bin_centers, slope * final_log_bin_centers + intercept, 'r-', linestyle='dashed', label=f'Fit line. Slope: {slope:.2f}')
    plt.title(f'Degree Histogram of a Homophilial B. A. Graph with β: {beta}')
    plt.xlabel('Normalized Degree Logarithm')
    plt.ylabel('Frecuency Logarithm')
    plt.grid(True)
    plt.legend()
    plt.xlim(None, -0.5)  # Set xlim to end at -0.5
    plt.show()

------